In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
!pip install tensorflow-estimator==1.15.2

     |████████████████████████████████| 502 kB 4.3 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 1.15.2 requires gast==0.2.2, but you have gast 0.5.3 which is incompatible.
tensorflow 1.15.2 requires tensorflow-estimator==1.15.1, but you have tensorflow-estimator 1.15.2 which is incompatible.
kapre 0.3.7 requires tensorflow>=2.0.0, but you have tensorflow 1.15.2 which is incompatible.


In [ ]:
import numpy as np
import pickle
import cv2
from os import listdir
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer
from keras.models import Sequential
from keras.models import load_model
from keras.layers import BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img
from keras.optimizers import Adam, SGD, Adagrad
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from keras.models import Model
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications.xception import Xception
from keras.layers import Conv2D, AveragePooling2D, GlobalAveragePooling2D
#import cv2
import tensorflow.keras
import os
#tf.logging.set_verbosity(tf.logging.ERROR)


Using TensorFlow backend.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
default_image_size = tuple((64,64))
image_size = 0
directory_root = '/content/drive/MyDrive/plantvillage'
width=64
height=64
depth=3

In [ ]:
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            return cv2.resize(image, default_image_size)
            #return img_to_array(image)
        else:
            return np.array([])
        
    except Exception as e:
        print("Error : ".format(e))
        return None

In [ ]:
import sys
print(sys.version_info)

sys.version_info(major=3, minor=7, micro=13, releaselevel='final', serial=0)


In [ ]:
image_list, label_list =[],[]
try:
    print("Loading images ...")
    root_dir = listdir(directory_root)
    for directory in root_dir:
        if directory == ".DS_Store" :
            root_dir.remove(directory)
            
    for plant_folder in root_dir :
        #print('gh')
#         plant_disease_folder_list = listdir(" %(root)s/%(sub)s/" %{'root':directory_root, 'sub':plant_folder})
        plant_disease_folder_list = listdir(directory_root+'/'+plant_folder)
        #print(plant_disease_folder_list)
        
        for disease_folder in plant_disease_folder_list:
            if disease_folder == ".DS_Store" :
                plant_disease_folder_list.remove(disease_folder)
                
        for plant_disease_folder in plant_disease_folder_list:
            print("Processing %s ..." %plant_disease_folder)
            plant_disease_image_list = listdir(directory_root+"/"+plant_folder+"/"+plant_disease_folder+"/")
            #print('ghll')
            
            for single_plant_disease_image in plant_disease_image_list:
                if single_plant_disease_image == ".DS_store":
                    plant_disease_image_list.remove(single_plant_disease_image)
            
            for image in plant_disease_image_list:
                image_directory =directory_root+"/"+plant_folder+"/"+plant_disease_folder+"/"+image
                #print(image_directory)
                if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True :
                    image_list.append(convert_image_to_array(image_directory))
                    label_list.append(plant_disease_folder)
                    
    print("--- Loading images completed ---")
    #print(label_list)
    
except Exception as e:
    print("Error : ".format(e))

Loading images ...
Processing Pepper__bell___Bacterial_spot ...
Processing Pepper__bell___healthy ...
Processing Potato___Early_blight ...
Processing Potato___Late_blight ...
Processing Potato___healthy ...
Processing Tomato_Bacterial_spot ...
Processing Tomato_Early_blight ...
Processing Tomato_Late_blight ...
Processing Tomato_Leaf_Mold ...
Processing Tomato_Septoria_leaf_spot ...
Processing Tomato_Spider_mites_Two_spotted_spider_mite ...
Processing Tomato__Target_Spot ...
Processing Tomato__Tomato_YellowLeaf__Curl_Virus ...
Processing Tomato__Tomato_mosaic_virus ...
Processing Tomato_healthy ...
--- Loading images completed ---


In [ ]:
image_size = len(image_list)

In [ ]:
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
#pickle.dump(label_binarizer, open('label_transform.pk1','wb'))
n_classes = len(label_binarizer.classes_)

print(label_binarizer.classes_)
#print

['Pepper__bell___Bacterial_spot' 'Pepper__bell___healthy'
 'Potato___Early_blight' 'Potato___Late_blight' 'Potato___healthy'
 'Tomato_Bacterial_spot' 'Tomato_Early_blight' 'Tomato_Late_blight'
 'Tomato_Leaf_Mold' 'Tomato_Septoria_leaf_spot'
 'Tomato_Spider_mites_Two_spotted_spider_mite' 'Tomato__Target_Spot'
 'Tomato__Tomato_YellowLeaf__Curl_Virus' 'Tomato__Tomato_mosaic_virus'
 'Tomato_healthy']


In [ ]:
import tensorflow as tf
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)
with tf.Session() as sess:
    print(sess.run(c))

[[22. 28.]
 [49. 64.]]


In [ ]:

# from PIL import Image
# for i in range(10):
#     image_list[i].shape
#     img = Image.fromarray(image_list[i], 'RGB')
#     img.save('my'+str(i)+'.png')

In [ ]:
np_image_list = np.array(image_list, dtype=np.float16)/225.0 #255?

In [ ]:
print("Splitting data")
x_train, x_test, y_train, y_test= train_test_split(np_image_list, image_labels, test_size =0.2, random_state=42)

Splitting data


In [ ]:
print(len(x_train))
print(x_train.shape)
print(n_classes)

16508
(16508, 64, 64, 3)
15


In [ ]:
aug = ImageDataGenerator(
    rotation_range=5, width_shift_range=0.1,
    height_shift_range = 0.1,
    horizontal_flip = True,
    )

In [ ]:
# aug = ImageDataGenerator(
#     rotation_range=0, width_shift_range=0,
#     height_shift_range = 0, shear_range=0,
#     zoom_range=0, horizontal_flip = True,
#     fill_mode='nearest')

In [ ]:
model = Sequential()
inputShape = (height, width, depth)
chanDim = -1
if K.image_data_format() == "channels_first":
    inputShape = (depth, height, width)
    chanDim= 1
    
model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
model.add(BatchNormalization(axis=chanDim))
model.add(Activation("relu"))

model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding="same"))
model.add(BatchNormalization(axis=chanDim))
model.add(Activation("relu"))

model.add(Conv2D(64, (3, 3), padding="same"))
model.add(BatchNormalization(axis=chanDim))
model.add(Activation("relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding="same"))
model.add(BatchNormalization(axis=chanDim))
model.add(Activation("relu"))

model.add(Conv2D(128, (3, 3), padding="same"))
model.add(BatchNormalization(axis=chanDim))
model.add(Activation("relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.4))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.4))

model.add(Dense(n_classes))
model.add(Activation("softmax"))

model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 21, 21, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 21, 21, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 21, 21, 64)        18496     
______________________________________

In [ ]:
n_classes

15

In [ ]:
# # base_model = VGG16(weights='imagenet', include_top=False)
# #base_model = Alex(weights='imagenet', include_top=False)
# base_model = InceptionV3(weights='imagenet', include_top=False, input_shape = (64,64,3) )
# #base_model = Xception(include_top=False, weights='imagenet')
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# # let's add a fully-connected layer
# x = Dense(75, activation='relu')(x)
# y = Dropout(0.3)(x)
# # and a logistic layer -- let's say we have 10 classes
# predictions = Dense(n_classes, activation='softmax')(y)

# model = Model(inputs=base_model.input, outputs=predictions)
# print(base_model.input)
# # for layser in model.layers:
# #    layer.trainable = True
# model.summary()

In [ ]:
# incept = tensorflow.keras.applications.inception_v3.InceptionV3(include_top=False, weights='imagenet', input_tensor=None, input_shape=None, pooling=None)#, classes=1000)

In [ ]:
EPOCHS = 30
INIT_LR = 0.001
BS = 32
#opt = SGD(lr=INIT_LR, decay=1e-6, momentum=0.9, nesterov=True)
#opt = Adagrad(lr=INIT_LR)
opt=Adam(lr=INIT_LR)#, decay=INIT_LR / EPOCHS)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])

print("Training...")

Training...


In [ ]:
#y_test

In [ ]:
history = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_test, y_test),
#    steps_per_epoch=len(x_train) /BS,
    steps_per_epoch=40,
    epochs=EPOCHS, verbose=1
    )


Epoch 1/30
40/40 [==============================] - 7s 171ms/step - loss: 2.3820 - accuracy: 0.2891 - val_loss: 3.2288 - val_accuracy: 0.0766
Epoch 2/30
40/40 [==============================] - 2s 54ms/step - loss: 1.8222 - accuracy: 0.4203 - val_loss: 4.6387 - val_accuracy: 0.0766
Epoch 3/30
40/40 [==============================] - 2s 54ms/step - loss: 1.5178 - accuracy: 0.5102 - val_loss: 8.4846 - val_accuracy: 0.0766
Epoch 4/30
40/40 [==============================] - 2s 54ms/step - loss: 1.3798 - accuracy: 0.5555 - val_loss: 10.3587 - val_accuracy: 0.0766
Epoch 5/30
40/40 [==============================] - 2s 53ms/step - loss: 1.2188 - accuracy: 0.6102 - val_loss: 7.2015 - val_accuracy: 0.1512
Epoch 6/30
40/40 [==============================] - 2s 54ms/step - loss: 1.1676 - accuracy: 0.6266 - val_loss: 9.5156 - val_accuracy: 0.1347
Epoch 7/30
40/40 [==============================] - 2s 54ms/step - loss: 1.1348 - accuracy: 0.6398 - val_loss: 7.0385 - val_accuracy: 0.1705
Epoch 8/30

In [ ]:
def step_decay(epoch):
    initial_lrate=0.001
    drop=0.1
    epochs_drop=16.0
    lrate=initial_lrate * math.pow(drop,math.floor((1+epoch)/epochs_drop))
    return lrate

In [ ]:
data_aug = False
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001,decay = 1e-03),metrics=['accuracy'])

if not data_aug:
    print('Not using real-time data augmentation.')
    history = model.fit(x_train,y_train,validation_data=(x_test,y_test),batch_size=8,epochs=15,verbose=2,steps_per_epoch=None)
    

else:
    print('Using real-time data augmentation.')
    lr=tensorflow.keras.callbacks.LearningRateScheduler(step_decay)#,verbose=1)
   # This will do preprocessing and realtime data augmentation:
#     tbCallback=tensorflow.keras.callbacks.TensorBoard(log_dir='./Logs/Tensorboard_Logs', histogram_freq=0, 
#                                            batch_size=32, write_graph=False, write_grads=False, 
#                                            write_images=False, embeddings_freq=0, embeddings_layer_names=None, 
#                                            embeddings_metadata=None)
#     checkpoint=tensorflow.keras.callbacks.ModelCheckpoint('./Logs/Checkpoint', monitor='val_loss', 
#                                                verbose=0, save_best_only=False, 
#                                                save_weights_only=False, 
#                                                mode='auto',
#                                                period=1)

    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=180,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.5,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.5,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True)  # randomly flip images


#     Compute quantities required for feature-wise normalization
#     (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    history = model.fit_generator(datagen.flow(x_train, y_train,batch_size=16),steps_per_epoch=20,epochs=10, validation_data=(x_test,y_test),workers=4)



Not using real-time data augmentation.
Train on 16508 samples, validate on 4128 samples
Epoch 1/15
 - 26s - loss: 0.8066 - accuracy: 0.7390 - val_loss: 0.3771 - val_accuracy: 0.8796
Epoch 2/15
 - 24s - loss: 0.5368 - accuracy: 0.8246 - val_loss: 0.3923 - val_accuracy: 0.8643
Epoch 3/15
 - 24s - loss: 0.4258 - accuracy: 0.8581 - val_loss: 0.1853 - val_accuracy: 0.9467
Epoch 4/15
 - 24s - loss: 0.3820 - accuracy: 0.8781 - val_loss: 0.2529 - val_accuracy: 0.9208
Epoch 5/15
 - 24s - loss: 0.3459 - accuracy: 0.8858 - val_loss: 0.1875 - val_accuracy: 0.9331
Epoch 6/15
 - 24s - loss: 0.3312 - accuracy: 0.8905 - val_loss: 0.1723 - val_accuracy: 0.9450
Epoch 7/15
 - 23s - loss: 0.2903 - accuracy: 0.9042 - val_loss: 0.1371 - val_accuracy: 0.9566
Epoch 8/15
 - 23s - loss: 0.2857 - accuracy: 0.9075 - val_loss: 0.1467 - val_accuracy: 0.9540
Epoch 9/15
 - 23s - loss: 0.2790 - accuracy: 0.9084 - val_loss: 0.1247 - val_accuracy: 0.9610
Epoch 10/15
 - 24s - loss: 0.2645 - accuracy: 0.9123 - val_loss: 0

In [ ]:
print(tf.__version__)

1.15.2


In [ ]:
#print("Saving model...")
#pickle.dump(model,open('cnn_model.pk1','wb'))
model.save('my_model.h5') 